In [6]:
#converting website to pdfkit

# # import pdfkit
# # pdfkit.from_file('https://flexbooks.ck12.org/book/ck-12-earth-science-for-middle-school/section/16.1/', '16_1.pdf')

# from pyhtml2pdf import converter

# converter.convert('https://flexbooks.ck12.org/book/ck-12-earth-science-for-middle-school/section/17.3/', '17_3.pdf')

In [62]:
AZURE_OPENAI_ENDPOINT = "https://xmm.openai.azure.com/"
AZURE_OPENAI_KEY = ""
TEXT_SEARCH_DOC_EMBEDDING_ENGINE = "text-search-curie-doc-001"
TEXT_SEARCH_QUERY_EMBEDDING_ENGINE = "text-search-curie-query-001"
TEXT_DAVINCI = "text-davinci-003"
GTPTurbo = "gpt-35-turbo"
COG_SEARCH_RESOURCE = "cog-search"
COG_SEARCH_KEY = "bxAzSeCYhKkh"
COG_SEARCH_INDEX = "index"
STORAGE_CONNECTION_STRING = "e.windows.net"
STORAGE_ACCOUNT = "storage"
STORAGE_CONTAINER = "mycontin"
STORAGE_KEY = "blah"
COG_SERVICE_KEY = "key"
DEBUG = "1"
functionAppUrlAndKey = "https://mappulrlqis7AzFugTuQ6Q=="

## Cog Search

1. Create datasource
2. Create Skillset
3. Create Index
4. Create Indexer

In [39]:
import requests
import re 
from transformers import GPT2TokenizerFast
import openai
import os
import json
from tenacity import retry, wait_random_exponential, stop_after_attempt
from openai.embeddings_utils import get_embedding, cosine_similarity
from langchain.vectorstores import FAISS
from langchain import FAISS
from langchain.llms import AzureOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
import numpy as np
import pandas as pd

In [40]:
def create_datasource(service_name, index_name, search_api_key, storage_connectionstring, storage_container):
    endpoint = "https://{}.search.windows.net/".format(service_name)


    url = '{0}/datasources/{1}-datasource?api-version=2020-06-30'.format(endpoint, index_name)

    print(url)
    payload = json.dumps({
                "description": "Demo files to demonstrate cognitive search capabilities.",
                "type": "azureblob",
                "credentials": {
                    "connectionString": storage_connectionstring
                },
                "container": {
                    "name": storage_container
                }
                })
    headers = {
    'api-key': search_api_key,
    'Content-Type': 'application/json'
            }


    response = requests.request("PUT", url, headers=headers, data=payload)

    if response.status_code == 201 or response.status_code == 204:
        return response, True
    else:
        return response, False
    
def create_skillset(service_name, index, cognitive_search_key, cognitive_service_key, embeddingFunctionAppUriAndKey):
    endpoint = "https://{}.search.windows.net/".format(service_name)
    url = '{0}/skillsets/{1}-skillset?api-version=2021-04-30-Preview'.format(endpoint, index)
    print(url)
    payload = json.dumps({
    "@odata.context": "{}/$metadata#skillsets/$entity".format(endpoint),
    "@odata.etag": "\"0x8DB2B4BF82370CF\"",
    "name": "{0}-skillset".format(index),
    "description": "Skillset created from the portal. skillsetName: index-skillset; contentField: merged_content; enrichmentGranularity: document; knowledgeStoreStorageAccount: ;",
    "skills": [
        {
        "@odata.type": "#Microsoft.Skills.Text.V3.EntityRecognitionSkill",
        "name": "#1",
        "description": None,
        "context": "/document/merged_content",
        "categories": [
            "Organization",
            "URL",
            "DateTime",
            "Skill",
            "Address",
            "Location",
            "Product",
            "IPAddress",
            "Event",
            "Person",
            "Quantity",
            "PersonType",
            "PhoneNumber",
            "Email"
        ],
        "defaultLanguageCode": "en",
        "minimumPrecision": None,
        "modelVersion": None,
        "inputs": [
            {
            "name": "text",
            "source": "/document/merged_content"
            },
            {
            "name": "languageCode",
            "source": "/document/language"
            }
        ],
        "outputs": [
            {
            "name": "persons",
            "targetName": "people"
            },
            {
            "name": "organizations",
            "targetName": "organizations"
            },
            {
            "name": "locations",
            "targetName": "locations"
            }
        ]
        },
        {
        "@odata.type": "#Microsoft.Skills.Text.KeyPhraseExtractionSkill",
        "name": "#2",
        "description": None,
        "context": "/document/merged_content",
        "defaultLanguageCode": "en",
        "maxKeyPhraseCount": None,
        "modelVersion": None,
        "inputs": [
            {
            "name": "text",
            "source": "/document/merged_content"
            },
            {
            "name": "languageCode",
            "source": "/document/language"
            }
        ],
        "outputs": [
            {
            "name": "keyPhrases",
            "targetName": "keyphrases"
            }
        ]
        },
        {
        "@odata.type": "#Microsoft.Skills.Text.LanguageDetectionSkill",
        "name": "#3",
        "description": None,
        "context": "/document",
        "defaultCountryHint": None,
        "modelVersion": None,
        "inputs": [
            {
            "name": "text",
            "source": "/document/merged_content"
            }
        ],
        "outputs": [
            {
            "name": "languageCode",
            "targetName": "language"
            }
        ]
        },
        {
        "@odata.type": "#Microsoft.Skills.Text.TranslationSkill",
        "name": "#4",
        "description": None,
        "context": "/document/merged_content",
        "defaultFromLanguageCode": None,
        "defaultToLanguageCode": "en",
        "suggestedFrom": "en",
        "inputs": [
            {
            "name": "text",
            "source": "/document/merged_content"
            }
        ],
        "outputs": [
            {
            "name": "translatedText",
            "targetName": "translated_text"
            }
        ]
        },
        {
        "@odata.type": "#Microsoft.Skills.Text.PIIDetectionSkill",
        "name": "#5",
        "description": None,
        "context": "/document/merged_content",
        "defaultLanguageCode": "en",
        "minimumPrecision": 0.5,
        "maskingMode": "replace",
        "maskingCharacter": "*",
        "modelVersion": None,
        "piiCategories": [],
        "domain": "none",
        "inputs": [
            {
            "name": "text",
            "source": "/document/merged_content"
            },
            {
            "name": "languageCode",
            "source": "/document/language"
            }
        ],
        "outputs": [
            {
            "name": "piiEntities",
            "targetName": "pii_entities"
            },
            {
            "name": "maskedText",
            "targetName": "masked_text"
            }
        ]
        },
        {
        "@odata.type": "#Microsoft.Skills.Text.MergeSkill",
        "name": "#6",
        "description": None,
        "context": "/document",
        "insertPreTag": " ",
        "insertPostTag": " ",
        "inputs": [
            {
            "name": "text",
            "source": "/document/content"
            },
            {
            "name": "itemsToInsert",
            "source": "/document/normalized_images/*/text"
            },
            {
            "name": "offsets",
            "source": "/document/normalized_images/*/contentOffset"
            }
        ],
        "outputs": [
            {
            "name": "mergedText",
            "targetName": "merged_content"
            }
        ]
        },
        {
        "@odata.type": "#Microsoft.Skills.Vision.OcrSkill",
        "name": "#7",
        "description": None,
        "context": "/document/normalized_images/*",
        "textExtractionAlgorithm": None,
        "lineEnding": "Space",
        "defaultLanguageCode": "en",
        "detectOrientation": True,
        "inputs": [
            {
            "name": "image",
            "source": "/document/normalized_images/*"
            }
        ],
        "outputs": [
            {
            "name": "text",
            "targetName": "text"
            },
            {
            "name": "layoutText",
            "targetName": "layoutText"
            }
        ]
        },
        {
        "@odata.type": "#Microsoft.Skills.Vision.ImageAnalysisSkill",
        "name": "#8",
        "description": None,
        "context": "/document/normalized_images/*",
        "defaultLanguageCode": "en",
        "visualFeatures": [
            "tags",
            "description"
        ],
        "details": [],
        "inputs": [
            {
            "name": "image",
            "source": "/document/normalized_images/*"
            }
        ],
        "outputs": [
            {
            "name": "tags",
            "targetName": "imageTags"
            },
            {
            "name": "description",
            "targetName": "imageCaption"
            }
        ]
        }
        ,
        {
  "@odata.type": "#Microsoft.Skills.Custom.WebApiSkill",
  "uri": embeddingFunctionAppUriAndKey,
  "httpMethod": "POST",
  "timeout": "PT230S",
  "batchSize": 1,
  "degreeOfParallelism": 1,
  "name": "Embeddings",
  "description": "",
  "context": "/document",
  "inputs": [
        {
          "name": "text",
          "source": "/document/merged_content"
        }
  ],
  "outputs": [
          {
            "name": "embeddings",
            "targetName": "embeddings"
          },
					{
            "name": "embeddings_text",
            "targetName": "embeddings_text"
          }
  ]
}

    ],
    "cognitiveServices": {
        "@odata.type": "#Microsoft.Azure.Search.CognitiveServicesByKey",
        "description": "SuperCool",
        "key": "{0}".format(cognitive_service_key)
    },
    "knowledgeStore": None,
    "encryptionKey": None
    })
    
    headers = {
        'Content-Type': 'application/json',
        'api-key': '{0}'.format(cognitive_search_key)
    }

    
    response = requests.request("PUT", url, headers=headers, data=payload)

    print(response.text)

    if response.status_code == 201 or response.status_code == 204:
        return response, True
    else:

        return response, False
    
def update_index_semantic(service_name, index, cognitive_search_key):

    endpoint = "https://{}.search.windows.net/".format(service_name)
    url = '{0}/indexes/{1}/?api-version=2021-04-30-Preview'.format(endpoint, index)
    print(url)

    payload = json.dumps({
    "name": index,
    "defaultScoringProfile": "",
    "fields": [
        {
        "name": "content",
        "type": "Edm.String",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "metadata_storage_content_type",
        "type": "Edm.String",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_storage_size",
        "type": "Edm.Int64",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_storage_last_modified",
        "type": "Edm.DateTimeOffset",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_storage_content_md5",
        "type": "Edm.String",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_storage_name",
        "type": "Edm.String",
        "searchable": False,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_storage_path",
        "type": "Edm.String",
        "searchable": False,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": True,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_storage_file_extension",
        "type": "Edm.String",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None, 
        "synonymMaps": []
        },
        {
        "name": "metadata_content_type",
        "type": "Edm.String",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_language",
        "type": "Edm.String",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "metadata_creation_date",
        "type": "Edm.DateTimeOffset",
        "searchable": False,
        "filterable": False,
        "retrievable": False,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": None,
        "synonymMaps": []
        },
        {
        "name": "people",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "organizations",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "locations",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "keyphrases",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "language",
        "type": "Edm.String",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "translated_text",
        "type": "Edm.String",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "en.lucene",
        "synonymMaps": []
        },
        {
        "name": "embeddings_text",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "embeddings",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "pii_entities",
        "type": "Collection(Edm.ComplexType)",
        "fields": [
            {
            "name": "text",
            "type": "Edm.String",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "standard.lucene",
            "synonymMaps": []
            },
            {
            "name": "type",
            "type": "Edm.String",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "standard.lucene",
            "synonymMaps": []
            },
            {
            "name": "subtype",
            "type": "Edm.String",
            "searchable": True,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": "standard.lucene",
            "synonymMaps": []
            },
            {
            "name": "offset",
            "type": "Edm.Int32",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
            },
            {
            "name": "length",
            "type": "Edm.Int32",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
            },
            {
            "name": "score",
            "type": "Edm.Double",
            "searchable": False,
            "filterable": False,
            "retrievable": True,
            "sortable": False,
            "facetable": False,
            "key": False,
            "indexAnalyzer": None,
            "searchAnalyzer": None,
            "analyzer": None,
            "synonymMaps": []
            }
        ]
        },
        {
        "name": "masked_text",
        "type": "Edm.String",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "merged_content",
        "type": "Edm.String",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "text",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "layoutText",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "imageTags",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        },
        {
        "name": "imageCaption",
        "type": "Collection(Edm.String)",
        "searchable": True,
        "filterable": False,
        "retrievable": True,
        "sortable": False,
        "facetable": False,
        "key": False,
        "indexAnalyzer": None,
        "searchAnalyzer": None,
        "analyzer": "standard.lucene",
        "synonymMaps": []
        }
    ],
    "scoringProfiles": [],
    "corsOptions": None,
    "suggesters": [],
    "semantic": {
        "defaultConfiguration": None,
        "configurations": [
        {
            "name": "semanic-config",
            "prioritizedFields": {
            "titleField": {
                "fieldName": "metadata_storage_name"
            },
            "prioritizedContentFields": [
                {
                "fieldName": "merged_content"
                }
            ],
            "prioritizedKeywordsFields": [
                {
                "fieldName": "keyphrases"
                },
                {
                "fieldName": "people"
                },
                {
                "fieldName": "locations"
                }
            ]
            }
        }
        ]
    },
    "analyzers": [],
    "tokenizers": [],
    "tokenFilters": [],
    "charFilters": [],
    "encryptionKey": None,
    "similarity": {
        "@odata.type": "#Microsoft.Azure.Search.BM25Similarity",
        "k1": None,
        "b": None
    }
    })
    headers = {
    'api-key': cognitive_search_key,
    'Content-Type': 'application/json'
    }

    response = requests.request("PUT", url, headers=headers, data=payload)

    if response.status_code == 201 or response.status_code == 204:
        return response, True
    else:
        # print('************************')
        # print(response.status_code)
        # print(response.text)
        return response, False

def create_indexer(service_name, index, search_key):
    endpoint = "https://{}.search.windows.net/".format(service_name)
    url = '{0}/indexers/{1}-indexer/?api-version=2021-04-30-Preview'.format(endpoint, index)
    print(url)

    payload = json.dumps({
    "name": "{0}-indexer".format(index),
    "description": "",
    "dataSourceName": "{0}-datasource".format(index),
    "skillsetName": "{0}-skillset".format(index),
    "targetIndexName": "{0}".format(index),
    "disabled": None,
    "schedule": None,
    "parameters": {
        "batchSize": None,
        "maxFailedItems": 0,
        "maxFailedItemsPerBatch": 0,
        "base64EncodeKeys": None,
        "configuration": {
        "dataToExtract": "contentAndMetadata",
        "parsingMode": "default",
        "imageAction": "generateNormalizedImages"
        }
    },
    "fieldMappings": [
        {
        "sourceFieldName": "metadata_storage_path",
        "targetFieldName": "metadata_storage_path",
        "mappingFunction": {
            "name": "base64Encode",
            "parameters": None
        }
        }
    ],
    "outputFieldMappings": [
        {
        "sourceFieldName": "/document/merged_content/people",
        "targetFieldName": "people"
        },
        {
        "sourceFieldName": "/document/merged_content/organizations",
        "targetFieldName": "organizations"
        },
        {
        "sourceFieldName": "/document/merged_content/locations",
        "targetFieldName": "locations"
        },
        {
        "sourceFieldName": "/document/merged_content/keyphrases",
        "targetFieldName": "keyphrases"
        },
        {
        "sourceFieldName": "/document/language",
        "targetFieldName": "language"
        },
        {
        "sourceFieldName": "/document/merged_content/translated_text",
        "targetFieldName": "translated_text"
        },
        {
        "sourceFieldName": "/document/merged_content/pii_entities",
        "targetFieldName": "pii_entities"
        },
        {
        "sourceFieldName": "/document/merged_content/masked_text",
        "targetFieldName": "masked_text"
        },
        {
        "sourceFieldName": "/document/merged_content",
        "targetFieldName": "merged_content"
        },
        {
        "sourceFieldName": "/document/normalized_images/*/text",
        "targetFieldName": "text"
        },
        {
        "sourceFieldName": "/document/normalized_images/*/layoutText",
        "targetFieldName": "layoutText"
        },
        {
        "sourceFieldName": "/document/normalized_images/*/imageTags/*/name",
        "targetFieldName": "imageTags"
        },
        {
        "sourceFieldName": "/document/normalized_images/*/imageCaption",
        "targetFieldName": "imageCaption"
        },
        {
        "sourceFieldName": "/document/embeddings",
        "targetFieldName": "embeddings"
        },
        {
        "sourceFieldName": "/document/embeddings_text",
        "targetFieldName": "embeddings_text"
        }
        
    ],
    "cache": None,
    "encryptionKey": None
    })
    headers = {
    'Content-Type': 'application/json',
    'api-key': '{0}'.format(search_key)
    }


    response = requests.request("PUT", url, headers=headers, data=payload)


    if response.status_code == 201 or response.status_code == 204:
        print('good')
        return response, True
    else:
        print(response.status_code)
        return response, False

In [41]:
response, success = create_datasource(COG_SEARCH_RESOURCE, COG_SEARCH_INDEX, COG_SEARCH_KEY, STORAGE_CONNECTION_STRING, STORAGE_CONTAINER)
print(response)
if success == True:
    print(functionAppUrlAndKey)
    response, success = create_skillset(COG_SEARCH_RESOURCE, COG_SEARCH_INDEX, COG_SEARCH_KEY, COG_SERVICE_KEY, functionAppUrlAndKey)

    print(response)
if success == True:
    response, success =  update_index_semantic(COG_SEARCH_RESOURCE, COG_SEARCH_INDEX, COG_SEARCH_KEY)
    print(response)
if success == True:
    response, success =  create_indexer(COG_SEARCH_RESOURCE, COG_SEARCH_INDEX, COG_SEARCH_KEY)
    print(response)

https://mmx-cog-search.search.windows.net//datasources/indexberks-datasource?api-version=2020-06-30
<Response [201]>
https://mygithubfuncghwqt.azurewebsites.net/api/Embeddings?code=1tt12p5jXZXmjRdze67JVr2fUqlbgtSOMv563Uydqis7AzFugTuQ6Q==
https://mmx-cog-search.search.windows.net//skillsets/indexberks-skillset?api-version=2021-04-30-Preview
{"@odata.context":"https://mmx-cog-search.search.windows.net/$metadata#skillsets/$entity","@odata.etag":"\"0x8DB63741FDDBE76\"","name":"indexberks-skillset","description":"Skillset created from the portal. skillsetName: index-skillset; contentField: merged_content; enrichmentGranularity: document; knowledgeStoreStorageAccount: ;","skills":[{"@odata.type":"#Microsoft.Skills.Text.V3.EntityRecognitionSkill","name":"#1","description":null,"context":"/document/merged_content","categories":["Organization","URL","DateTime","Skill","Address","Location","Product","IPAddress","Event","Person","Quantity","PersonType","PhoneNumber","Email"],"defaultLanguageCode"

In [63]:
def get_api_key():
    openai.api_type = "azure"
    openai.api_base = "https://.openai.azure.com/"
    openai.api_version = "2022-12-01"
    os.environ['OPENAI_API_KEY'] = ""
    openai.api_key = os.getenv("OPENAI_API_KEY")


#@retry(wait=wait_random_exponential(multiplier=0.5,  max=60))
def langChainIt(prompt, full_input_text):

    print('starting langChainIt')
    get_api_key()
    # #create FAISS
    embeddings = OpenAIEmbeddings()
    faiss = FAISS.from_texts(full_input_text, embeddings)
    docs_db = faiss.similarity_search(prompt)
    llm = AzureOpenAI(deployment_name="text-davinci-003-deployment", model_name="text-davinci-003-deployment", temperature=0.5, max_tokens=2000) 
    chain = load_qa_chain(llm, chain_type="refine")
    print('loading chain')
    response = chain({"input_documents": docs_db, "question": prompt, "language": "English", "existing_answer" : ""}, return_only_outputs=True)
    return(response['output_text'])

def gettoken_count(document):
    tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
    try:
        token_count = len(tokenizer.encode(document))
    except:
        print('failed to get token count')
        token_count = -1
        pass

    return token_count

def normalize_text(s, sep_token = " \n "):
    s = re.sub(r'\s+',  ' ', s).strip()
    s = re.sub(r". ,","",s)
    s = s.replace("..",".")
    s = s.replace(". .",".")
    s = s.replace("\n", "")
    s = s.strip()
    
    return s

def search_semantic(service_name, index, search_key,  question):
    if len(question) > 0:
        endpoint = "https://{}.search.windows.net/".format(service_name)
        url = '{0}indexes/{1}/docs/search?api-version=2021-04-30-Preview'.format(endpoint, index)

        print(url)

        payload = json.dumps({
        "search": question,
        "queryType": "semantic",
        "queryLanguage": "en-us",
        "captions": "extractive",
        "answers": "extractive",
        "semanticConfiguration": "semanic-config",
        "count": True,
        })
        headers = {
        'api-key': '{0}'.format(search_key),
        'Content-Type': 'application/json'
        }

        response = requests.request("POST", url, headers=headers, data=payload)
 
        obj = response.json()

        
        try:
            print('answer found')
            answer = obj['@search.answers'][0]['text']
        except:
            answer = obj['value'][0]['@search.captions'][0]['text']
            pass
        
        try:
            embeddings_text = []
            embeddings = []
            count_data = obj['@odata.count']
            relevant_data = obj['value'][0]['content']
            file_name = obj['value'][0]['metadata_storage_name']
            embeddings_text = obj['value'][0]['embeddings_text']
            
            embeddings = obj['value'][0]['embeddings']
        except:
            count_data = 0
            relevant_data = ''
            file_name = 'none'

        return answer, relevant_data, count_data, file_name, embeddings_text, embeddings
    else:
        return "", "", 0, '', [], []
    
@retry(wait=wait_random_exponential(multiplier=0.5,  max=60))
def get_answer(embeddings, embeddings_text, question):
    get_api_key()
    curie_search = []
    for x in embeddings:
        a = np.fromstring(x[1:-1], dtype=float, sep=',')
        curie_search.append(a)
    curie_list = list(curie_search)
    np_vectors = np.array(curie_search).astype('float32')
    df = pd.DataFrame(list(zip(embeddings_text, curie_list)),
                columns =['text', 'curie_search'])
    response = openai.Embedding.create(
        input=question,
        engine="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']
    df["similarities"] = df.curie_search.apply(lambda x: cosine_similarity(x, embeddings))
    res = (df.sort_values("similarities", ascending=False).reset_index(drop=True).head(1))
    context = res['text'][0]
    prompt_engineering=f"Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:"
    try:
        # Create a completions using the question and context
        prompt_token_count = gettoken_count(prompt_engineering)
        response_token_count = 4000 - prompt_token_count
        response = openai.Completion.create(
        engine="text-davinci-003-deployment",
        prompt=prompt_engineering, #question + "\nTl;dr",
        temperature=0.3,
        max_tokens=response_token_count,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        best_of=1,
        stop=None)
        return response.choices[0].text

    except Exception as e:
        print('An exception has occured.')
        print(e)
        return ""


In [52]:

def getmyanswer(question):
  answer, relevant_data, count_data, file_name, embeddings_text, embeddings = search_semantic(COG_SEARCH_RESOURCE, COG_SEARCH_INDEX, COG_SEARCH_KEY,  question)
  context = normalize_text(relevant_data)
  prompt_engineering=f"Answer the question based on the context below, and if the question can't be answered based on the context, say \"I don't know\"\n\nContext: {context}\n\n---\n\nQuestion: {question}\nAnswer:"
  total_token_count = gettoken_count(context)
  print(total_token_count)
  result = get_answer(embeddings, embeddings_text, question)
  print(result)
  return result, context

In [61]:
import time

time.sleep(60)
question = "what is the address for Robert H. Bird?"
result, context = getmyanswer(question)



https://mmx-cog-search.search.windows.net/indexes/indexberks/docs/search?api-version=2021-04-30-Preview
answer found


Token indices sequence length is longer than the specified maximum sequence length for this model (8549 > 1024). Running this sequence through the model will result in indexing errors


8549
 I don't know.


In [57]:
print(context)

url: https://berkshirehathaway.com/letters/1981.html Chairman's Letter - 1981 BERKSHIRE HATHAWAY INC. February 26, 1982 To the Shareholders of Berkshire Hathaway Inc.: Operating earnings of $39.7 million in 1981 amounted to 15.2% of beginning equity capital (valuing securities at cost) compared to 17.8% in 1980. Our new plan that allows stockholders to designate corporate charitable contributions (detailed later) reduced earnings by about $900,000 in 1981. This program, which we expect to continue subject to annual evaluation of our corporate tax position, had not been initiated in 1980. Non-Controlled Ownership Earnings In the 1980 annual report we discussed extensively the concept of non-controlled ownership earnings, i.e., Berkshire�s share of the undistributed earnings of companies we don�t control or significantly influence but in which we, nevertheless, have important investments. (We will be glad to make available to new or prospective shareholders copies of that discussion or o